<center><img src="logo_skmob.png" width=450 align="left" /></center>

# Flow models

- Repo: [http://bit.ly/skmob_repo](http://bit.ly/skmob_repo)
- Docs: [http://bit.ly/skmob_doc](http://bit.ly/skmob_doc)
- Paper: [http://bit.ly/skmob_paper](http://bit.ly/skmob_paper)

# Outline

#### Goal: Train a Gravity model in San Francisco and use it to predict the flows in Denver

1. Download checkin data and create a `TrajDataFrame`
1. Create square tessellations for the two cities
1. Compute the relevance (total number of visits) of each tile
1. Create `FlowDataFrame`s for the two cities aggregating the trips in the `TrajDataFrame` that are inside the two regions

4. Compute the total number of trips from each tile (we want to predict the destinations of these trips)
1. Fit a singly-constrained Gravity Model using the trips in San Francisco
1. Use the fitted model to predict the flows in Denver
1. Qualitative evaluation: visualise the performance of the model against a baseline random model
1. Quantitative evaluation: compute various performance metrics

### Import libraries we need

In [1]:
import skmob
from skmob.utils import utils, constants
from skmob.tessellation import tilers
from skmob.utils.plot import plot_gdf

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import folium
from folium.plugins import HeatMap
import matplotlib as mpl
import matplotlib.pyplot as plt

# Download Brightkite checkin data and create a `TrajDataFrame`

In [2]:
url = "https://snap.stanford.edu/data/loc-brightkite_totalCheckins.txt.gz"
df = pd.read_csv(url, sep='\\t', header=0, nrows=100000, 
                 names=['user', 'check-in_time', 'latitude', 'longitude', 'location id'])

/home/lucpappalard/miniconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
tdf = skmob.TrajDataFrame(df, latitude='latitude', longitude='longitude', datetime='check-in_time', user_id='user')
print(len(tdf))
tdf.head()

100000


,uid,datetime,lat,lng,location id
0,0,2010-10-16 06:02:04+00:00,39.891383,-105.070814,7a0f88982aa015062b95e3b4843f9ca2
1,0,2010-10-16 03:48:54+00:00,39.891077,-105.068532,dd7cd3d264c2d063832db506fba8bf79
2,0,2010-10-14 18:25:51+00:00,39.750469,-104.999073,9848afcc62e500a01cf6fbf24b797732f8963683
3,0,2010-10-14 00:21:47+00:00,39.752713,-104.996337,2ef143e12038c870038df53e0478cefc
4,0,2010-10-13 23:31:51+00:00,39.752508,-104.996637,424eb3dd143292f9e013efa00486c907


## Visualise the raw trajectory data of 10 users


In [4]:
tdf.plot_trajectory(max_users=10, max_points=1000, zoom=4, start_end_markers=False)

## Visualise a heat map of checkin locations


In [5]:
m = folium.Map(tiles = 'openstreetmap', zoom_start=12, control_scale=True)
HeatMap(tdf[:50000][['lat', 'lng']].values).add_to(m)
m

# Create square tessellations for the two regions (cities)

## Train

In [6]:
tess_train = tilers.tiler.get("squared", meters=2500, base_shape="San Francisco, California")
len(tess_train)

207

## Test

In [7]:
tess_test = tilers.tiler.get("squared", meters=2500, base_shape="Denver, Colorado")
len(tess_test)

158

# Compute the relevance (total number of visits) of each tile


In [8]:
tdf_tid = tdf.mapping(tess_train, remove_na=True)
relevances = tdf_tid.groupby(by='tile_ID').count()[['lat']].rename(columns={'lat': 'relevance'})
# normalise
relevances /= relevances.sum()

tess_train = tess_train.merge(relevances, right_index=True, left_on='tile_ID', how='left').fillna(0.)
tess_train[:3]

,tile_ID,geometry,relevance
0,0,"POLYGON ((-123.173825 37.72917858254642, -123....",0.0
1,1,"POLYGON ((-123.173825 37.74693866076989, -123....",0.0
2,2,"POLYGON ((-123.173825 37.76469447796389, -123....",0.0


In [9]:
tdf_tid = tdf.mapping(tess_test, remove_na=True)
relevances = tdf_tid.groupby(by='tile_ID').count()[['lat']].rename(columns={'lat': 'relevance'})
# normalise
relevances /= relevances.sum()

tess_test = tess_test.merge(relevances, right_index=True, left_on='tile_ID', how='left').fillna(0.)
tess_test[:3]

,tile_ID,geometry,relevance
0,0,"POLYGON ((-105.1098845 39.6143154, -105.109884...",0.00047
1,1,"POLYGON ((-105.1098845 39.63161375638285, -105...",0.00000
2,2,"POLYGON ((-105.087426617897 39.6143154, -105.0...",0.00000


## Visualise the relevance of each tile on a map

In [10]:
def define_colormap(tessellation, minval=1e-6):
    # define the colormap
    normc = mpl.colors.LogNorm(vmin=max(tessellation['relevance'].min(), minval), \
                               vmax=tessellation['relevance'].max())
    s_m = mpl.cm.ScalarMappable(cmap='jet', norm=normc)
    return s_m

def get_color(x):
    return mpl.colors.to_hex(s_m.to_rgba(x['relevance'] + 1e-12))

In [11]:
s_m = define_colormap(tess_train)

plot_gdf(tess_train, zoom=10, popup_features=['relevance'], \
         style_func_args={'color': get_color, 'fillColor' : get_color})

In [12]:
s_m = define_colormap(tess_test)

plot_gdf(tess_test, zoom=10, popup_features=['relevance'], \
         style_func_args={'color': get_color, 'fillColor' : get_color})

# Create `FlowDataFrame`s for the two cities aggregating the trips in the `TrajDataFrame` that are inside the two regions

In [13]:
fdf_train = tdf.to_flowdataframe(tess_train, remove_na=True, self_loops=False)
print(fdf_train['flow'].sum(), fdf_train['flow'].max())
# print(len(fdf_train))
fdf_train[:4]

2305 177


,origin,destination,flow
1,111,137,1
2,111,148,1
4,112,137,1
5,112,146,2


In [14]:
fdf_test = tdf.to_flowdataframe(tess_test, remove_na=True, self_loops=False)
print(fdf_test['flow'].sum(), fdf_test['flow'].max())
# print(len(fdf_test))
fdf_test[:4]

10627 445


,origin,destination,flow
1,0,127,1
2,0,34,1
3,0,35,1
4,0,4,1


In [15]:
fdf_train.plot_flows(min_flow=5, zoom=10, tiles='cartodbpositron', flow_weight=2, opacity=0.25)

In [16]:
fdf_test.plot_flows(min_flow=5, zoom=10, tiles='cartodbpositron', flow_weight=2, opacity=0.25)

# Compute the total number of trips from each tile (we want to predict the destinations of these trips)

In [17]:
# total outflows excluding self loops
tot_outflows = fdf_train[fdf_train['origin'] != fdf_train['destination']] \
    .groupby(by='origin', axis=0)[['flow']].sum().fillna(0).rename(columns={'flow': 'tot_outflow'})

tess_train = tess_train.merge(tot_outflows, right_index=True, left_on='tile_ID', how='left').fillna(0.)
tess_train[:4]

,tile_ID,geometry,relevance,tot_outflow
0,0,"POLYGON ((-123.173825 37.72917858254642, -123....",0.0,0.0
1,1,"POLYGON ((-123.173825 37.74693866076989, -123....",0.0,0.0
2,2,"POLYGON ((-123.173825 37.76469447796389, -123....",0.0,0.0
3,3,"POLYGON ((-123.173825 37.78244603344593, -123....",0.0,0.0


In [18]:
# total outflows excluding self loops
tot_outflows = fdf_test[fdf_test['origin'] != fdf_test['destination']] \
    .groupby(by='origin', axis=0)[['flow']].sum().fillna(0).rename(columns={'flow': 'tot_outflow'})

tess_test = tess_test.merge(tot_outflows, right_index=True, left_on='tile_ID', how='left').fillna(0.)
tess_test[:4]

,tile_ID,geometry,relevance,tot_outflow
0,0,"POLYGON ((-105.1098845 39.6143154, -105.109884...",0.00047,8.0
1,1,"POLYGON ((-105.1098845 39.63161375638285, -105...",0.00000,0.0
2,2,"POLYGON ((-105.087426617897 39.6143154, -105.0...",0.00000,0.0
3,3,"POLYGON ((-105.087426617897 39.63161375638285,...",0.00000,0.0


# Fit a singly-constrained Gravity Model using the trips in San Francisco

In [19]:
from skmob.models.gravity import Gravity

## Fit the gravity model's parameters


In [20]:
gravity_singly_fitted = Gravity(gravity_type='singly constrained')
print(gravity_singly_fitted)

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.0], origin_exp=1.0, destination_exp=1.0, gravity_type="singly constrained")


In [21]:
gravity_singly_fitted.fit(fdf_train, relevance_column='relevance')
print(gravity_singly_fitted)


Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-0.8323557156771302], origin_exp=1.0, destination_exp=0.709313443150735, gravity_type="singly constrained")


# Use the fitted model to predict the flows in Denver

In [23]:
np.random.seed(0)
scGM_fdf_fitted = gravity_singly_fitted.generate(tess_test, 
                                   tile_id_column='tile_ID', 
                                   tot_outflows_column='tot_outflow', 
                                   relevance_column= 'relevance',
                                   out_format='flows')
scGM_fdf_fitted[:4]

100%|██████████| 158/158 [00:00<00:00, 2003.13it/s]


ValueError: pvals < 0, pvals > 1 or pvals contains NaNs

In [24]:
scGM_fdf_fitted.plot_flows(min_flow=5, zoom=10, tiles='cartodbpositron', flow_weight=2, opacity=0.25)

NameError: name 'scGM_fdf_fitted' is not defined

# Qualitative evaluation: visualise the performance of the model against a baseline random model

## Create a baseline model (without dependence on relevance and distance)

In [25]:
baseline = Gravity(gravity_type='singly constrained', deterrence_func_args=[0.], destination_exp=0.)
print(baseline)

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[0.0], origin_exp=1.0, destination_exp=0.0, gravity_type="singly constrained")


In [26]:
np.random.seed(0)
baseline_fdf = baseline.generate(tess_test, 
                                   tile_id_column='tile_ID', 
                                   tot_outflows_column='tot_outflow', 
                                   relevance_column= 'relevance',
                                   out_format='flows')
baseline_fdf[:4]

100%|██████████| 158/158 [00:00<00:00, 2007.67it/s]


,origin,destination,flow
0,0,34,1
1,0,42,2
2,0,43,2
3,0,48,1


In [27]:
baseline_fdf.plot_flows(min_flow=5, zoom=10, tiles='cartodbpositron', flow_weight=2, opacity=0.25)

## Compare real flows against flows generated by the models

In [28]:
xy = fdf_test.merge(scGM_fdf_fitted, on=['origin', 'destination'])[['flow_x', 'flow_y']].values
xy_baseline = fdf_test.merge(baseline_fdf, on=['origin', 'destination'])[['flow_x', 'flow_y']].values

NameError: name 'scGM_fdf_fitted' is not defined

In [29]:
plt.plot(xy[:,0], xy[:,1], '.', label='Gravity')
plt.plot(xy_baseline[:,0], xy_baseline[:,1], '*', alpha=0.5, label='Baseline')

x = np.logspace(0, np.log10(np.max(xy)))
plt.plot(x, x, '--k')

plt.xlabel('Real flow')
plt.ylabel('Model flow')
plt.legend(loc = 'upper left')

plt.loglog()
plt.show()

NameError: name 'xy' is not defined

# Quantitative evaluation metrics


In [30]:
from skmob.measures.evaluation import r_squared, mse, spearman_correlation, pearson_correlation, common_part_of_commuters, common_part_of_commuters_distance

In [ ]:
metrics = [r_squared, mse, spearman_correlation, pearson_correlation, common_part_of_commuters, common_part_of_commuters_distance]
names = ['r_squared', 'mse', 'spearman_correlation', 'pearson_correlation', 'common_part_of_commuters', 'common_part_of_commuters_distance']

print('Metric:  Gravity - Baseline')
print('---------------------------')
for i, metric in enumerate(metrics):
    m = metric(xy[:, 0], xy[:, 1])
    b = metric(xy_baseline[:, 0], xy_baseline[:, 1])
    print("%s:   %s - %s" % (names[i], np.round(m, 3), np.round(b, 3)))
